# Nipah Virus Inhibitor Discovery Pipeline
**Student:** Vihaan Agrawal  
**Project:** Computational Discovery of Resistance-Proof Antivirals

## Welcome
This notebook is the **Unified Control Center** for my research project. It allows you to:
1.  **Prepare the Data:** Download the virus structure and create the mutant from scratch.
2.  **Run the Simulation:** Test the drug candidates using the Vina physics engine.
3.  **Verify the Results:** Analyze the data and prove the resistance profile.

---

## Phase 0: Environment Setup (UV)
We use **uv** for fast, reliable dependency management. This cell ensures all libraries are installed.

In [ ]:
import sys
import subprocess

print("Checking Dependencies via uv...")
subprocess.run(["uv", "pip", "install", "-r", "requirements.txt"], check=True)
print("Dependencies Verified.")

## Phase 1: Data Preparation ("The Kitchen")
Before we can run any physics, we need to correct the raw biological data. This involves cleaning the PDB file and creating the W730A mutant.

In [ ]:
import os
from pathlib import Path

# Define paths to our specialized background scripts
BASE_DIR = Path(".").resolve()
SCRIPTS_DIR = BASE_DIR / "scripts"
PIPELINE_SCRIPT = SCRIPTS_DIR / "pipeline.py"
SETUP_SCRIPT = SCRIPTS_DIR / "setup.py"
VERIFY_SCRIPT = SCRIPTS_DIR / "verify.py"

if not SCRIPTS_DIR.exists():
    print("Error: 'scripts/' folder missing! check directory.")
else:
    print("Environment Found. Ready to run.")

### Step 1.1: Run the 'From Scratch' Build
This command runs `scripts/setup.py`, which:
1.  **Downloads** 9KNZ.cif from the Protein Data Bank.
2.  **Cleans** the water molecules.
3.  **Mutates** W730 to Alanine.
4.  **Converts** to PDBQT format.

In [ ]:
print("Starting Building Process... (This might take a minute)")
subprocess.run(["python3", str(SETUP_SCRIPT)], check=True)
print("Build Complete. Data is in 'data/' folder.")

---
## Phase 2: The Experiments ("The Lab")
Now that the receptors are ready, we run the verification experiment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def run_pipeline(ligand_file):
    print(f"Processing {ligand_file}...")
    # Note: We invoke python3 directly, assuming we are inside the active venv/conda env
    result = subprocess.run(
        ["python3", str(PIPELINE_SCRIPT), str(ligand_file)], 
        capture_output=True, 
        text=True
    )
    print(result.stdout)
    return result.stdout

### Step 2.1: Test the Lead Candidate (BMS-986205)

In [ ]:
bms_output = run_pipeline("data/ligand_BMS_986205.pdbqt")

### Step 2.2: Test the Control (ERDRP-0519)

In [ ]:
erdrp_output = run_pipeline("data/ligand_ERDRP_0519.pdbqt")

---
## Phase 3: Final Verification & Visualization
We pull the aggregate stats to confirm robustness and plot the final Resistance Profile.

In [ ]:
print("Running Statistical Verification (5 Seeds)...")
# This runs verify.py which does the heavy lifting
subprocess.run(["python3", str(VERIFY_SCRIPT)], check=True)

### Visualization
Generating the Resistance Confidence Graph.

In [ ]:
# Data from our Verified Findings (Hardcoded from verify.py AVG outputs for clean plotting)
# In a full dynamic system, verify.py could dump a JSON to read here.
# For this notebook, we visualize the confirmed values.

labels = ['BMS-986205', 'ERDRP-0519']
wt_means = [-7.501, -6.69]
mut_means = [-7.501, -6.52]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(8,6))
rects1 = ax.bar(x - width/2, wt_means, width, label='Wild Type', color='#2ca02c')
rects2 = ax.bar(x + width/2, mut_means, width, label='Mutant (W730A)', color='#d62728')

# Add text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Binding Affinity (kcal/mol)')
ax.set_title('Resistance Profile: Lead vs Control')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Invert Y axis because lower energy is better
ax.set_ylim(-8.0, 0)

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

# Save graph to disk
plt.savefig("resistance_graph.png", dpi=300)
plt.show()
print("Graph saved to resistance_graph.png")